In [1]:
#######################
# ベテランルート抽出 #
######################
import xlrd
import numpy as np

DEBUG_PRINT_NUM=10

# Excel読み込み
wb_2019_06 = xlrd.open_workbook('data/2019_06.xls')
sheet = wb_2019_06.sheets()[0]
wb_enter_date = xlrd.open_workbook('data/EnterDate.xlsx')
sheet_enter_date = wb_enter_date.sheets()[0]

# 緯度/経度 取得
latitude_list = sheet.col_values(19)
longitude_list = sheet.col_values(20)

# 配送日付 取得
date_list_1 = sheet.col_values(1)
date_list_2 = sheet.col_values(12)

date_list = date_list_1 # 持ち出し日付を配送日付とする

# 配送時刻 取得
time_list = sheet.col_values(13)

# ドライバーID 取得
driver_list = sheet.col_values(6)
driver_list_2 = sheet_enter_date.col_values(0)
enter_date_list = sheet_enter_date.col_values(1)

veteran_driver_list = []

veteran_num = 0
my_id = 1
my_driver_list = [0 for i in range(999999)]
for _driver_id in driver_list[1:]:
    driver_id = int(_driver_id)
    my_driver_id = my_driver_list[driver_id]
    if driver_id != 0 and my_driver_id == 0:
        my_driver_list[driver_id] = my_id
        my_id += 1
        for i, driver_id_2 in enumerate(driver_list_2[1:]):
            if i != 0:
                enter_date = int(enter_date_list[i])
                year = int(enter_date / 10000)
            if (int(driver_id_2) == driver_id) and (year <= 16 or 80 <= year):
                veteran_driver_list.append(driver_id)
                veteran_num += 1
            

print('総ドライバー数={0}'.format(my_id))
print('ベテランドライバー数={0}'.format(veteran_num))
print('ベテランドライバー数（veteran_driver_listの要素数）={0}'.format(len(veteran_driver_list)))

# 取得データをまとめる
raw_data_num = len(date_list)
valid_data_num = 0
index_max = 0

valid_data = np.ones((my_id + 1, 32, 100, 3))

def check_data(driver_id, lat, lon): 
    return ((driver_id != 0) and (lat != 0.0) and (lon != 0.0) and (driver_id in veteran_driver_list))
    #return ((driver_id != 0) and (lat != 0.0) and (lon != 0.0))

for i in range(1, raw_data_num):
    driver_id = int(my_driver_list[int(driver_list[i])])
    date = int(date_list[i]%100)
    # (緯度, 経度, 配送時刻)の3つを格納データとする
    _valid_data = [float(latitude_list[i]), float(longitude_list[i]), int(time_list[i])]
    
    if check_data(int(driver_list[i]), _valid_data[0], _valid_data[1]):
        # ドライバーID, 配送日付, インデックス(データを受け取った順番) によって振り分け，リストに格納
        index = 0
        while int(valid_data[driver_id][date][index][0]) != 1:
            index += 1
        # 時刻が離れているかどうかの判定（近い場合は同じ配送先と判断し，データを格納しない）
        if index == 0 or abs(valid_data[driver_id][date][index-1][2]%100 - _valid_data[2]%100) > 1:
            valid_data[driver_id][date][index] = _valid_data
            
            index_max = max(index, index_max)
            valid_data_num += 1

#################################################################################################
print('上から{0}個のデータを見てみる\n'.format(DEBUG_PRINT_NUM))

print('\n\n 座標')
print('----------------------------------------')
for i in range(DEBUG_PRINT_NUM):
    print('{0}: ({1}, {2})'.format(i, latitude_list[i], longitude_list[i]))
print('----------------------------------------')
print('\n\n 日付')
print('----------------------------------------')
for i in range(DEBUG_PRINT_NUM):
    print('{0}: ({1}, {2})'.format(i, date_list_1[i], date_list_2[i]))
print('----------------------------------------')
print('\n\n 持出日付と完了日付が一致していないデータ')
print('----------------------------------------')
for i in range(1, len(date_list_1)):
    date1 = date_list_1[i]
    date2 = date_list_2[i]
    if date1 != date2:
        print('(持出日付, 完了日付)=({0}, {1})'.format(date1, date2))
print('\n\n 時刻')
print('----------------------------------------')
for i in range(DEBUG_PRINT_NUM):
    print('{0}: {1}'.format(i, time_list[i]))
print('----------------------------------------')
print('\n\n ドライバーID')
print('----------------------------------------')
for i in range(DEBUG_PRINT_NUM):
    print('{0}: {1}'.format(i, driver_list[i]))
print('----------------------------------------')
print('\n\n ベテランドライバーID')
print('----------------------------------------')
for i in range(DEBUG_PRINT_NUM):
    print('{0}: {1}'.format(i, veteran_driver_list[i]))
print('----------------------------------------')
sample_id = 222911
sample_date = 20
print('\n\n ドライバーID={0}さんの2019年6月{1}日における配送データ'.format(sample_id, sample_date))
print('----------------------------------------')
for index, sample_data in enumerate(valid_data[my_driver_list[sample_id]][sample_date][:]):
    if int(sample_data[0]) != 1:
        print('{0}: {1}'.format(index, sample_data))
print('----------------------------------------')

print('\n\n 2019年6月の総データ数={0}'.format(raw_data_num))
print('ドライバーIDが欠損していないデータ数={0}'.format(valid_data_num))
print('最大配送回数={0}'.format(index_max))
#################################################################################################



総ドライバー数=107
ベテランドライバー数=87
ベテランドライバー数（veteran_driver_listの要素数）=87
上から10個のデータを見てみる



 座標
----------------------------------------
0: (緯度＿完了優先, 経度＿完了優先)
1: (35.098673, 136.6362308)
2: (35.3055556, 136.601572)
3: (35.3055556, 136.601572)
4: (35.3055556, 136.601572)
5: (35.3055556, 136.601572)
6: (35.3901153, 136.5810473)
7: (35.0949862, 136.6695377)
8: (35.3046609, 136.6171996)
9: (35.3046609, 136.6171996)
----------------------------------------


 日付
----------------------------------------
0: (持出日付, 完了日付)
1: (20190620.0, 20190620.0)
2: (20190607.0, 20190607.0)
3: (20190611.0, 20190611.0)
4: (20190612.0, 20190612.0)
5: (20190613.0, 20190613.0)
6: (20190610.0, 20190610.0)
7: (20190603.0, 20190603.0)
8: (20190606.0, 20190606.0)
9: (20190620.0, 20190620.0)
----------------------------------------


 持出日付と完了日付が一致していないデータ
----------------------------------------


 時刻
----------------------------------------
0: 完了時刻
1: 1422.0
2: 1034.0
3: 958.0
4: 1017.0
5: 1120.0
6: 1117.0
7: 923.0
8: 1040.